In [22]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
from querychains import with_context, Context, OpenAiEngine, start_server
import json

def show(obj):
    print(json.dumps(obj, indent=2))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Minimal context
with Context("MyContext") as c:
    pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "be6aacb8-70ee-4e84-b9ed-6ee487fedd5c"
}


In [13]:
# Context nesting
with Context("MyContext") as c:
    with Context("ChildContext1"):
        pass
    with Context("ChildContext2"):
        pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "9ee3d5f8-a108-44a5-9ed6-f122fe779c92",
  "children": [
    {
      "_type": "Context",
      "name": "ChildContext1",
      "uuid": "bf0ebe9e-37e4-4779-af54-b3b392c6fa2d"
    },
    {
      "_type": "Context",
      "name": "ChildContext2",
      "uuid": "f4acad23-049f-45e5-b33d-15493a77555a"
    }
  ]
}


In [14]:
# Inputs and outputs
with Context("MyContext", inputs={"a": 10, "b": 20}) as c:
    c.set_result(30)

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "770a953b-ec6a-40a7-9640-6921367618fb",
  "inputs": {
    "a": 10,
    "b": 20
  },
  "result": 30
}


In [15]:
# Context decorator
@with_context
def my_function(a, b):
    return a + b

with Context("root") as c:
    my_function(10, 20)
    
show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "c60b9b38-c8b7-41cd-a18c-e1a229c19c39",
  "children": [
    {
      "_type": "Context",
      "name": "my_function",
      "uuid": "379b8893-67c3-4d53-a346-588164114681",
      "kind": "call",
      "inputs": {
        "a": 10,
        "b": 20
      },
      "result": 30
    }
  ]
}


In [16]:
# Error handling

@with_context
def my_function(a, b):
    raise Exception("Oops")

try:
    with Context("root") as c:
        my_function(10, 20)
except:
    pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "838b5acb-ef41-4811-8d0e-d848cda2a6ec",
  "state": "error",
  "error": {
    "_type": "error",
    "name": "Oops"
  },
  "children": [
    {
      "_type": "Context",
      "name": "my_function",
      "uuid": "58b45be2-6ac2-45dc-9e48-98f61fa3b857",
      "state": "error",
      "kind": "call",
      "inputs": {
        "a": 10,
        "b": 20
      },
      "error": {
        "_type": "error",
        "name": "Oops"
      }
    }
  ]
}


In [17]:
# Dataclasses

@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "0472e68c-535e-48a5-8bb5-c67f0d4cb9b3",
  "children": [
    {
      "_type": "Context",
      "name": "say_hi",
      "uuid": "7c42fbbb-1038-440d-9084-4671c334f0a0",
      "kind": "call",
      "inputs": {
        "person": {
          "name": "Alice",
          "age": 21,
          "_type": "Person"
        }
      },
      "result": "Hi Alice!"
    }
  ]
}


In [18]:
# Data browser

server = start_server()  # Writes an URL where the server is running

INFO:     Started server process [15533]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:38841 (Press CTRL+C to quit)


INFO:     127.0.0.1:51854 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:51854 - "GET /static/js/main.ef9f1307.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:51868 - "GET /static/css/main.073c9b0a.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:51854 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:51872 - "GET /manifest.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51868 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:51868 - "GET /logo192.png HTTP/1.1" 200 OK


In [21]:
@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

server.add_context(c)

INFO:     127.0.0.1:55156 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/uuid/20ea4dd9-d87e-461e-ab17-830aba97c060 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/uuid/b8edbc0f-27dd-4e59-89a6-65f4740bda71 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/uuid/5a2e80f0-0b31-4462-b89f-861802cbbeff HTTP/1.1" 200 OK


In [3]:
# LLM queries

engine = OpenAiEngine()

with Context("root") as c:
        response = engine.query("Hi are you?")
        engine.query("Is the following text genereted by LLM?\n\n" + response)
show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "9c4b98d3-e099-4031-808b-90ab938b2792",
  "children": [
    {
      "_type": "Context",
      "name": "OpenAiChat gpt-3.5-turbo",
      "uuid": "d5e79f60-e6b9-4ef3-95c8-39b705c18734",
      "kind": "query",
      "inputs": {
        "prompt": "Hi are you?",
        "conf": {
          "api": "OpenAiChat",
          "model": "gpt-3.5-turbo",
          "temperature": 0.7,
          "max_tokens": 1024,
          "_type": "QueryConf"
        }
      },
      "result": "As an AI language model, I don't have emotions, but I'm functioning well. How can I assist you today?"
    },
    {
      "_type": "Context",
      "name": "OpenAiChat gpt-3.5-turbo",
      "uuid": "53c008a5-5149-4768-a4fd-65150bff1d9a",
      "kind": "query",
      "inputs": {
        "prompt": "Is the following text genereted by LLM?\n\nAs an AI language model, I don't have emotions, but I'm functioning well. How can I assist you today?",
        "conf": {
          "api"